source: https://www.kaggle.com/tomras/cnn-classifier-using-keras

In [12]:
import json
import numpy as np
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Note: This notebook assumes keras-gpu is installed instead of the regular keras

Note: You'll need to downgrade to python 3.7 to install keras-gpu

Check for GPUs

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9073795663830306503
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6610781471
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17733127810773909071
physical_device_desc: "device: 0, name: GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


load data

In [3]:
f = open(r'./data/shipsnet.json')
dataset = json.load(f)
f.close()
data = np.array(dataset['data']).astype('uint8')
labels = np.array(dataset['labels']).astype('uint8')

normalize and split data

In [4]:
x = data / 255.
x = x.reshape([-1, 3, 80, 80]).transpose([0,2,3,1])
print(x.shape)
trainx=x[:3600]
y = to_categorical(labels, num_classes=2)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

(4000, 80, 80, 3)
(4000, 2)


define a basic CNN

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(80, 80, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
    
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), 
              metrics=['accuracy'])

train the model

In [6]:
history = model.fit(X_train, y_train, batch_size=32, epochs=40, validation_split=0.2)
score = model.evaluate(X_test, y_test) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Train on 2144 samples, validate on 536 samples
Epoch 1/40
2144/2144 [==============================] - 4s 2ms/step - loss: 0.3827 - accuracy: 0.8414 - val_loss: 0.2065 - val_accuracy: 0.9160uracy: 
Epoch 2/40
2144/2144 [==============================] - 2s 728us/step - loss: 0.2080 - accuracy: 0.9100 - val_loss: 0.1978 - val_accuracy: 0.91040s - loss: 0.2202 - accu - ETA: 0s - loss: 0.2221 - 
Epoch 3/40
2144/2144 [==============================] - 2s 724us/step - loss: 0.1528 - accuracy: 0.9370 - val_loss: 0.0963 - val_accuracy: 0.9608
Epoch 4/40
2144/2144 [==============================] - 2s 726us/step - loss: 0.1402 - accuracy: 0.9440 - val_loss: 0.0786 - val_accuracy: 0.9664
Epoch 5/40
2144/2144 [==============================] - 2s 728us/step - loss: 0.0763 - accuracy: 0.9711 - val_loss: 0.0627 - val_accuracy: 0.9813
Epoch 6/40
2144/2144 [==============================] - 2s 728us/step - loss: 0.0659 - accuracy: 0.9743 - val_loss: 0.0490 - val_accuracy: 0.9813
Epoch 7/40
2144/2144

create the model in a way that can be passed to a scikit wrapper for cross-validation

In [7]:
def create():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(80, 80, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), 
              metrics=['accuracy'])
    return model

run cross-validation

In [8]:
cnn = KerasClassifier(build_fn=create, epochs=40, batch_size=32,validation_split=.2)
scores=cross_val_score(cnn, X_train, y_train, cv=3)
print(np.mean(scores))

Train on 1428 samples, validate on 358 samples
Epoch 1/40
1428/1428 [==============================] - 2s 1ms/step - loss: 0.5045 - accuracy: 0.7605 - val_loss: 0.2587 - val_accuracy: 0.8911
Epoch 2/40
1428/1428 [==============================] - 1s 726us/step - loss: 0.2840 - accuracy: 0.8901 - val_loss: 0.1659 - val_accuracy: 0.9330
Epoch 3/40
1428/1428 [==============================] - 1s 725us/step - loss: 0.2102 - accuracy: 0.9118 - val_loss: 0.1505 - val_accuracy: 0.9469
Epoch 4/40
1428/1428 [==============================] - 1s 731us/step - loss: 0.1760 - accuracy: 0.9258 - val_loss: 0.1676 - val_accuracy: 0.9218
Epoch 5/40
1428/1428 [==============================] - 1s 736us/step - loss: 0.1382 - accuracy: 0.9517 - val_loss: 0.0931 - val_accuracy: 0.9721
Epoch 6/40
1428/1428 [==============================] - 1s 732us/step - loss: 0.1604 - accuracy: 0.9433 - val_loss: 0.1348 - val_accuracy: 0.9581
Epoch 7/40
1428/1428 [==============================] - 1s 735us/step - loss: 0

Epoch 15/40
1429/1429 [==============================] - 1s 745us/step - loss: 0.0267 - accuracy: 0.9909 - val_loss: 0.0528 - val_accuracy: 0.9777
Epoch 16/40
1429/1429 [==============================] - 1s 729us/step - loss: 0.0160 - accuracy: 0.9937 - val_loss: 0.0580 - val_accuracy: 0.9693
Epoch 17/40
1429/1429 [==============================] - 1s 723us/step - loss: 0.0114 - accuracy: 0.9958 - val_loss: 0.0664 - val_accuracy: 0.9777
Epoch 18/40
1429/1429 [==============================] - 1s 725us/step - loss: 0.0070 - accuracy: 0.9965 - val_loss: 0.0693 - val_accuracy: 0.9749
Epoch 19/40
1429/1429 [==============================] - 1s 729us/step - loss: 0.0129 - accuracy: 0.9958 - val_loss: 0.0534 - val_accuracy: 0.9832
Epoch 20/40
1429/1429 [==============================] - 1s 724us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0458 - val_accuracy: 0.9804
Epoch 21/40
1429/1429 [==============================] - 1s 720us/step - loss: 0.0061 - accuracy: 0.9972 - val_loss: 0

1429/1429 [==============================] - 1s 728us/step - loss: 8.2991e-04 - accuracy: 1.0000 - val_loss: 0.0937 - val_accuracy: 0.9777
Epoch 30/40
1429/1429 [==============================] - 1s 735us/step - loss: 4.3089e-04 - accuracy: 1.0000 - val_loss: 0.1024 - val_accuracy: 0.9777
Epoch 31/40
1429/1429 [==============================] - 1s 727us/step - loss: 0.0013 - accuracy: 0.9993 - val_loss: 0.1327 - val_accuracy: 0.9777
Epoch 32/40
1429/1429 [==============================] - 1s 735us/step - loss: 0.0083 - accuracy: 0.9986 - val_loss: 0.1185 - val_accuracy: 0.9749
Epoch 33/40
1429/1429 [==============================] - 1s 729us/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.0811 - val_accuracy: 0.9777
Epoch 34/40
1429/1429 [==============================] - 1s 728us/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.1064 - val_accuracy: 0.9777
Epoch 35/40
1429/1429 [==============================] - 1s 731us/step - loss: 0.0017 - accuracy: 0.9993 - val_loss: 0.121

double-check that test and validation scores are similar

In [9]:
cnn.fit(X_train,y_train)
cnn.score(X_test,y_test)

Train on 2144 samples, validate on 536 samples
Epoch 1/40
2144/2144 [==============================] - 2s 830us/step - loss: 0.4047 - accuracy: 0.8251 - val_loss: 0.2036 - val_accuracy: 0.9030oss: 0.4142 - accuracy: 0.82
Epoch 2/40
2144/2144 [==============================] - 2s 726us/step - loss: 0.2176 - accuracy: 0.9076 - val_loss: 0.1943 - val_accuracy: 0.9310
Epoch 3/40
2144/2144 [==============================] - 2s 728us/step - loss: 0.1620 - accuracy: 0.9342 - val_loss: 0.1204 - val_accuracy: 0.9459
Epoch 4/40
2144/2144 [==============================] - 2s 727us/step - loss: 0.1126 - accuracy: 0.9562 - val_loss: 0.0848 - val_accuracy: 0.9739
Epoch 5/40
2144/2144 [==============================] - 2s 729us/step - loss: 0.1079 - accuracy: 0.9599 - val_loss: 0.0667 - val_accuracy: 0.9720cura
Epoch 6/40
2144/2144 [==============================] - 2s 730us/step - loss: 0.0902 - accuracy: 0.9636 - val_loss: 0.0597 - val_accuracy: 0.9776
Epoch 7/40
2144/2144 [=======================

0.9840909242630005

try and fail to create a roc curve for this network

In [10]:

#from sklearn.metrics import roc_curve
#y_pred_keras = keras_model.predict(X_test)
#print(y_pred_keras)
#y_pred_keras=np.max(y_pred_keras,axis=1)
#print(y_test.shape)
#print(np.max(y_test,axis=1))
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(np.max(y_test,axis=1), y_pred_keras)
#from sklearn.metrics import auc
#auc_keras = auc(fpr_keras, tpr_keras)

In [11]:

#plt.figure(1)
#plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.xlabel('False positive rate')
#plt.ylabel('True positive rate')
#plt.title('ROC curve')
#plt.legend(loc='best')
#plt.show()